<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 2**
## **Кластеризация, сегментация и робастный анализ временных рядов и табличных данных**

---

### **1. Цель и задачи работы**

**Цель работы** — формирование у обучающегося системного представления о методологии предмодельного анализа внутренней структуры временных рядов и табличных данных, основанного на комплексном применении методов кластеризации, сегментации и робастного обнаружения аномалий. Работа направлена на освоение практических навыков диагностики гетерогенности данных, выявления скрытых режимов поведения, классификации и обработки выбросов, а также проектирования интерпретируемых признаков для последующего моделирования, при условии использования качественно подготовленного датасета из Практической работы № 1 и соблюдения строгих требований к воспроизводимости, отсутствию утечки будущего и научной обоснованности.

**Основные задачи работы:**
1. Использовать обработанный табличный датасет из Практической работы № 1, содержащий не менее 720 последовательных наблюдений, с подтверждённой стационарностью, стабилизированной дисперсией и богатым набором инженерных признаков (лаги, скользящие окна, циклические кодировки).
2. Провести многоуровневую диагностику выбросов с применением:
   – **статистических методов** (Z-score, Modified Z-score, IQR);
   – **машинных методов** (Isolation Forest, Local Outlier Factor, One-Class SVM);
   – **контекстно-зависимых методов** (STL + residuals, Twitter AnomalyDetection, Seasonal Hybrid ESD);
   – **визуальной верификации** и классификации аномалий на типы:
     • **Point anomaly** (одиночная ошибка),
     • **Contextual anomaly** (аномалия в контексте, например, высокий спрос в будний день),
     • **Collective anomaly** (аномальный сегмент, например, пандемия).
3. Реализовать и сравнить **три подхода к кластеризации временных рядов**:
   – **Feature-based**: кластеризация по статистическим и инженерным признакам (тренд, сезонность, автокорреляция, лаги);
   – **Shape-based**: DTW (Dynamic Time Warping), soft-DTW, k-Shape;
   – **Representation-based**: UMAP/TSNE над эмбеддингами, полученных через TS2Vec, TS-VAE или PCA.
4. Выполнить **сегментацию временного ряда** с использованием:
   – **PELT** (Pruned Exact Linear Time),
   – **Binary Segmentation**,
   – **Bayesian Online Changepoint Detection**,
   – **Window-based methods** (rolling window + statistical test).
5. Оценить качество кластеризации и сегментации по:
   – **внутренним метрикам**: Silhouette Score, Calinski–Harabasz Index, Davies–Bouldin Index;
   – **внешним метрикам**: бизнес-интерпретируемость, однородность автокорреляционной структуры, стационарность внутри сегментов.
6. Проанализировать профили кластеров и сегментов:
   – Среднее, дисперсия, автокорреляция, сезонность;
   – Распределение признаков;
   – Стационарность (ADF/KPSS);
   – ACF/PACF.
7. Разработать стратегии робастного моделирования:
   – Обучение отдельных моделей на кластерах/сегментах;
   – Добавление признака `cluster_id` или `segment_id`;
   – Использование робастных функций потерь (Huber, Quantile, ε-insensitive).
8. Реализовать **полный набор методов визуализации**:
   – Heatmap аномалий;
   – График ряда с маркировкой кластеров и точек смены;
   – t-SNE/UMAP с цветовой маркировкой;
   – Профильные диаграммы кластеров (boxplots, distributions);
   – ACF/PACF по сегментам.
9. Разработать интерактивный веб-инструмент для визуального анализа структуры данных.
10. Обеспечить воспроизводимость и открытость результатов: опубликовать код, сравнительные таблицы, веб-приложение и аналитический отчёт.

---

### **2. Теоретические предпосылки**

Временные ряды в реальных приложениях редко являются однородными. Часто они представляют собой **суперпозицию нескольких скрытых режимов** — например, «обычный спрос», «праздничный всплеск», «кризисная фаза». Игнорирование этой гетерогенности приводит к усреднённым, неадекватным моделям.

**Обработка выбросов** требует многоуровневого подхода:
– **Статистические методы** эффективны для грубых ошибок, но чувствительны к нестационарности;
– **Машинные методы** (Isolation Forest) хорошо работают в многомерном пространстве признаков;
– **Контекстно-зависимые методы** (STL, Twitter AD) учитывают сезонность и тренд, что критично для временных рядов.

**Кластеризация временных рядов** может осуществляться тремя принципиально разными способами:
– **Feature-based** — самый интерпретируемый и быстрый, подходит для предмоделирования;
– **Shape-based** — ориентирован на форму, устойчив к сдвигам во времени, требует выравнивания;
– **Representation-based** — наиболее гибкий, но менее интерпретируемый, требует обучения энкодера.

**Сегментация** (changepoint detection) — это задача обнаружения моментов, когда статистические свойства ряда резко меняются. Алгоритмы, такие как **PELT**, минимизируют функционал вида:
\[
\sum_{i=1}^m \left[ \mathcal{C}(y_{\tau_{i-1}+1:\tau_i}) + \beta \right],
\]
где \(\mathcal{C}\) — стоимость сегмента, \(\beta\) — штраф за новую точку смены.

Ключевой принцип: **анализ структуры данных — не предварительный этап, а часть моделирования**. Результаты кластеризации и сегментации напрямую влияют на выбор архитектуры, признаков и функции потерь в последующих работах.

---

### **3. Порядок выполнения работы**

#### **3.1. Использование данных из Практической работы № 1**

**Задача:** Обеспечить методологическую преемственность.
**Требования:**
– Датасет: ≥720 наблюдений, стационарность подтверждена, признаки включают лаги, скользящие окна, циклические кодировки;
– Сохранены параметры преобразований (λ, сдвиги);
– Временные метки в формате datetime.

#### **3.2. Многоуровневая диагностика выбросов**

**Задача:** Классифицировать и обработать аномалии.
**Требования:**
– Применить:
  • **IQR**: \( Q1 - 1.5 \cdot IQR,\ Q3 + 1.5 \cdot IQR \);
  • **Isolation Forest** с `contamination=auto`;
  • **STL-decomposition** → анализ остатков (3σ правило);
  • **Twitter AnomalyDetection** (параметры по умолчанию).
– Визуализация: временной ряд с маркерами аномалий (цвет = тип);
– Классификация: Point / Contextual / Collective;
– Для Point — интерполяция; для Collective — сегментация.

#### **3.3. Кластеризация временных рядов**

**Задача:** Выявить однородные группы.
**Требования:**
– **Feature-based**:
  • Признаки: `trend_30`, `seasonality_amp_7`, `lag_1_autocorr`, `rolling_std_30`;
  • Модели: K-Means, HDBSCAN, Gaussian Mixture;
– **Shape-based**:
  • Подряды длиной 60;
  • DTW-матрица → Agglomerative Clustering;
  • k-Shape (библиотека `tslearn`);
– **Representation-based**:
  • PCA → UMAP (n_components=2);
  • TSNE (perplexity=30);
  • Кластеризация в латентном пространстве.
– Оценка: Silhouette, Calinski–Harabasz, Davies–Bouldin;
– Выбор лучшего метода по комбинации метрик + интерпретируемости.

#### **3.4. Сегментация временной оси**

**Задача:** Найти точки смены режима.
**Требования:**
– Применить:
  • **PELT** (cost = `l2`, pen = 10);
  • **Bayesian Online CPD** ( hazard = 250);
  • **Binary Segmentation** (test = CUSUM).
– Визуализация: вертикальные линии на графике ряда;
– Анализ каждого сегмента: стационарность, дисперсия, тренд.

#### **3.5. Анализ профилей кластеров и сегментов**

**Задача:** Охарактеризовать выявленные структуры.
**Требования:**
– Для каждого кластера/сегмента:
  • Среднее, std, min, max;
  • ADF/KPSS (p-value);
  • ACF/PACF до лага 30;
  • Распределение `lag_1`, `rolling_mean_7`;
– Формирование текстового профиля:
  > «Кластер 2: высокая волатильность (σ = 12.4), сильная недельная сезонность, нестационарность (ADF p = 0.12)».


#### **3.6. Анализ интерпретируемости и устойчивости**

**Задача:** Оценить надёжность структур.
**Требования:**
– Повторить кластеризацию с 5 разными seed;
– Вычислить Adjusted Rand Index (ARI) между запусками;
– Анализ устойчивости точек смены: стандартное отклонение позиций.

#### **3.7. Методы визуализации**

**Задача:** Обеспечить полную визуальную диагностику.
**Требования:**
– **Обязательные графики**:
  1. Исходный ряд + аномалии (цвет = тип);
  2. Ряд + сегменты (цвет = сегмент);
  3. Heatmap признаков по кластерам;
  4. t-SNE/UMAP с цветовой маркировкой кластеров;
  5. Boxplots по признакам в кластерах;
  6. ACF/PACF по сегментам (side-by-side);
  7. Silhouette plot для K-Means;
  8. Профиль тренда и сезонности по кластерам.
– Все графики — интерактивные (Plotly).

#### **3.8. Разработка веб-интерфейса**

**Задача:** Создать инструмент для анализа.
**Требования:**
– Streamlit-приложение с вкладками:
  • «Аномалии»,
  • «Кластеры»,
  • «Сегменты»,
  • «Профили»,
  • «Сравнение моделей».
– Возможность загрузки CSV;
– Экспорт результатов в CSV/PDF.

#### **3.9. Публикация результатов**

**Задача:** Обеспечить воспроизводимость.
**Требования:**
– GitHub-репозиторий с:
  • `data/`, `notebooks/`, `src/`, `app/`;
  • `requirements.txt`, `Dockerfile`;
– Веб-приложение на Hugging Face Spaces;
– Лицензии: MIT (код), CC BY 4.0 (данные);
– Метаописание: методы, параметры, выводы.

---

### **4. Дополнительные исследовательские задания**

1. Сравнение DTW и soft-DTW по устойчивости к шуму.
2. Автоматический выбор числа кластеров: Silhouette + Gap Statistic.
3. Применение Wasserstein Distance для оценки диссимилярности кластеров.
4. Онлайн-сегментация: сравнение Bayesian CPD и PELT по latency.
5. Влияние выбросов на кластеризацию: эксперимент с/без очистки.
6. Интеграция кластеров в AutoML (работа № 4): как влияет `cluster_id` на AutoGluon?

---

### **5. Требования к отчёту**

Отчёт по ГОСТ 7.32–2017, с разделами:
1. Введение — актуальность анализа структуры данных;
2. Методология — все методы, метрики, инструменты;
3. Результаты — таблицы, графики, профили, сравнение стратегий;
4. Обсуждение — интерпретация кластеров, устойчивость, влияние на ML;
5. Заключение — рекомендации по использованию в production;
6. Список источников — ГОСТ Р 7.0.5–2008;
7. Приложения — код, скриншоты, схема pipeline.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Реализованы все 3 подхода к кластеризации + 4 метода сегментации + 4 метода аномалий; все метрики качества; полный набор визуализаций; профили кластеров; сравнение влияния на ML; функциональный веб-интерфейс; публикация; отчёт по ГОСТ. |
| **Хорошо** | Реализованы 2 подхода к кластеризации + 2 метода сегментации; есть метрики и визуализации; отсутствуют лишь отдельные элементы (например, soft-DTW или Bayesian CPD). |
| **Удовлетворительно** | Реализованы K-Means и PELT; базовая обработка выбросов; есть графики, но без профилей или сравнения. |
| **Неудовлетворительно** | Отсутствуют кластеризация или анализ аномалий; нет визуализаций; отчёт не предоставлен. |
